# Классификация комментариев

Интернет-магазину «Викишоп» нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.
Требуется обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.
Финальная модель должна быть со значением метрики качества F1 не меньше 0.75. 

In [1]:
%%capture
!pip install lightgbm -q
!pip install optuna -q
!pip install catboost
!pip install transformers -q
!pip install shutup -q
!pip install tensorflow_hub -q
!pip install -U pip
!pip install -U --no-cache-dir gdown --pre
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm

In [3]:
# %%capture
! gdown 1gAXKg1LRJOb9jk971V-m21gCxinTAy52  # эмбединги toxic-bert

Downloading...
From: https://drive.google.com/uc?id=1gAXKg1LRJOb9jk971V-m21gCxinTAy52
To: /content/df_tweets_embeddings_sl_cleaned_toxic_bert.pkl
100% 489M/489M [00:08<00:00, 60.7MB/s]


In [4]:
import time
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import re
import warnings
import torch
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.utils.vis_utils import plot_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, roc_curve
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tqdm.keras import TqdmCallback
from tqdm import notebook
from datetime import datetime
from io import BytesIO
from sklearn import preprocessing
from copy import deepcopy

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool
import tensorflow_hub as hub
import transformers
import shutup
import spacy
import optuna

from lightgbm import LGBMClassifier
from optuna.samplers import TPESampler
from transformers import AutoTokenizer

%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
# установка параметров системы
pd.options.display.max_columns = 100  # чтоб выводил все столбцы на экран
pd.options.display.float_format = (
    "{:,.3f}".format
)  # отображение количества знаков после запятой
optuna.logging.set_verbosity(optuna.logging.WARNING)
shutup.please()

In [6]:
try:
    df = pd.read_csv(
        "/datasets/toxic_comments.csv" , index_col=[0], parse_dates=[0])  # если нет интернета
except:
    print("ошибка чтения с диска")
try:
  df = pd.read_csv('toxic_comments.csv', index_col=[0], parse_dates=[0])
  df.shape  # проверка на существование датасета
except:
    try:
        df = pd.read_csv(
            "https://code.s3.yandex.net/datasets/toxic_comments.csv" , index_col=[0], parse_dates=[0]
        )  # если нет интернета
    except:
        print("ошибка связи с сетью")

df_raw = df.copy()  # сохраним нетронутые сеты на всякий случай
start_time_project = time.time()

ошибка чтения с диска


## Подготовка

Получим общую информацию о таблицах:

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Пропусков нет.

Выведем на экран первые пять строк таблиц:

In [9]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим данные на дубликаты:

In [10]:
print(df.duplicated().sum())

0


Дубликатов нет.

Данные готовы для дальнейшей работы.

Попробуем несколько вариантов по построению модели. 

Первым вариантом попробуем создать модель на основе TF-IDF.

## Обучение

### TF-IDF

Сначала проведем простую предобработку текста. Удалим стоп-слова и лишние символы:

In [11]:
df['clean_text'] = df['text'].str.lower()
stop_words = set(stopwords.words("english"))
new_stopwords = [',', ':', '&', '%', '’', 'utc']

sw = stop_words.union(new_stopwords)

In [12]:
def text_preprocessing(text):
  text = re.sub(r"['\-, ]", ' ', text)
  tokenized = nltk.word_tokenize(text)
  tokenized = [w for w in tokenized if not w in sw]
  joined = ' '.join(tokenized)
  text_only = re.sub(r"[^a-z!@#\$%\^\&\*_\-,\.' ]", ' ', joined)

  text_only = re.sub(r"['']", ' ', text_only)
  final = ' '.join(text_only.split())
  return final

In [13]:
%%time
df['clean_text'] = df['clean_text'].apply(text_preprocessing)
df.head()

CPU times: user 1min 22s, sys: 342 ms, total: 1min 22s
Wall time: 1min 24s


,text,toxic,clean_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...
1,D'aww! He matches this background colour I'm s...,0,aww ! matches background colour seemingly stuc...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war . guy constantl...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero . chance remember page


Далее, лемматизируем слова через Spacy.

In [14]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(text):
  doc = nlp(text)
  return " ".join([token.lemma_ for token in doc])

In [15]:
%%time
df['clean_text2'] = df['clean_text'].apply(lemmatize)

CPU times: user 9min 51s, sys: 3.1 s, total: 9min 55s
Wall time: 9min 52s


In [16]:
df.head()

,text,toxic,clean_text,clean_text2
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edit make username hardcore metall...
1,D'aww! He matches this background colour I'm s...,0,aww ! matches background colour seemingly stuc...,aww ! match background colour seemingly stuck ...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war . guy constantl...,hey man really try edit war . guy constantly r...
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...,make real suggestion improvement wonder sectio...
4,"You, sir, are my hero. Any chance you remember...",0,sir hero . chance remember page,sir hero . chance remember page


Объявим переменные:

In [17]:
rstate=12345
sampler = TPESampler(seed=10)  # фиксируем генератор пвсевдослучайности для optuna

nfolds = 5
trials = 25  # число итерации при оптимизации optuna


Разбиваем на выборки:

In [18]:
features = df['clean_text2'].copy()
target = df['toxic'].copy()

In [19]:
X_train, X_rem, y_train, y_rem = train_test_split(features,target,  test_size=0.3, 
                              random_state=rstate, shuffle=True, stratify=df['toxic']  )
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5,  random_state=rstate, shuffle=True, stratify=y_rem )

Трансформируем текст:

In [20]:
tf_idf = TfidfVectorizer() 
tf_idf.fit(X_train) 

X_train = tf_idf.transform(X_train) 
X_val = tf_idf.transform(X_val) 
X_test = tf_idf.transform(X_test) 

Нормализовать данные не будем, так как при Tf_idf векторизации данные имеют нормализованный вид.

#### LGBM 

Теперь обучим LGBM модель на преобразованных данных.

In [21]:
model = LGBMClassifier( n_estimators=5000,  random_seed=rstate, class_weight = 'balanced'    )
start = time.time()
model.fit(X_train, y_train,
          eval_set=[(X_val, y_val)], early_stopping_rounds=30,
          verbose=False, 
          )
end = time.time()
fit_time = end - start
print('fit time', fit_time)

fit time 717.7926440238953


Получим предсказания:

In [22]:
ppreds_val = model.predict_proba(X_val)

In [23]:
print('val roc_auc', round( roc_auc_score(y_val, ppreds_val[:,1]) , 3) )

val roc_auc 0.962


Получили очень приличное значение по roc_auc.

Подберем оптимальный порог нашей модели для получения максимального значения метрики f1 на валидационной выборке:


In [24]:
def objective(trial, val_preds):
    threshold = trial.suggest_float("threshold", 0, 1)
    preds = (val_preds[:,1]> threshold) * 1

    return f1_score(y_val, preds)

In [25]:
study = optuna.create_study(
    direction="maximize", study_name="tresh", sampler=sampler
)
func = lambda trial: objective(trial, ppreds_val)
study.optimize(func, n_trials=75, show_progress_bar=True)

  0%|          | 0/75 [00:00<?, ?it/s]

In [26]:
print(f"Лучший показатель f1 на валидационной выборке: {study.best_value:.5f}")
print("")
print(f"Лучший порог:")

for key, value in study.best_params.items():
    print(f"{key}: {value}")
    best_tresh = value

Лучший показатель f1 на валидационной выборке: 0.78351

Лучший порог:
threshold: 0.6389094766718714


Сохраним модель

In [27]:
lgbm = deepcopy(model)

#### LogisticRegression

Теперь попробуем на тех же данных обучить логистическую регрессию.

In [28]:
model = LogisticRegression( class_weight='balanced'   )
start = time.time()
model.fit(X_train, y_train    )
end = time.time()
fit_time = end - start
print('fit time', fit_time)

fit time 5.702531814575195


Получим предсказания:

In [29]:
ppreds_val = model.predict_proba(X_val)

In [30]:
print('val roc_auc', round( roc_auc_score(y_val, ppreds_val[:,1]) , 3) )

val roc_auc 0.971


Подберем оптимальный порог нашей модели для получения максимального значения метрики f1 на валидационной выборке:


In [31]:
study = optuna.create_study(
    direction="maximize", study_name="tresh", sampler=sampler
)
func = lambda trial: objective(trial, ppreds_val)
study.optimize(func, n_trials=75, show_progress_bar=True)

  0%|          | 0/75 [00:00<?, ?it/s]

In [32]:
print(f"Лучший показатель f1 на валидационной выборке: {study.best_value:.5f}")
print("")
print(f"Лучший порог:")

for key, value in study.best_params.items():
    print(f"{key}: {value}")
    best_tresh = value

Лучший показатель f1 на валидационной выборке: 0.78617

Лучший порог:
threshold: 0.6698298890062979


Roc_auc лучше чем у LGBM модели. Как и f1.

Сохраним модель:

In [33]:
lr = deepcopy(model)

### Catboost

В моделе градиентного бустинга Catboost есть возможность работы с текстом без его перевода в другой формат.

In [34]:
model = CatBoostClassifier(auto_class_weights='Balanced', 
    # task_type='GPU',
    iterations=5000,
    od_type='Iter',
    od_wait=30,
    text_processing = {
        "tokenizers" : [{
            "tokenizer_id" : "Space",
            "separator_type" : "ByDelimiter",
            "delimiter" : " "
        }],
    
        "dictionaries" : [{
            "dictionary_id" : "BiGram",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "2"
        }, {
            "dictionary_id" : "Word",
            "max_dictionary_size" : "50000",
            "occurrence_lower_bound" : "3",
            "gram_order" : "1"
        }],
    
        "feature_processing" : {
            "default" : [{
                "dictionaries_names" : ["BiGram", "Word"],
                "feature_calcers" : ["BoW"],
                "tokenizers_names" : ["Space"]
            }, {
                "dictionaries_names" : ["Word"],
                "feature_calcers" : ["NaiveBayes"],
                "tokenizers_names" : ["Space"]
            }],
        }
    }
)


Разделим данные на выборки:

In [35]:
text_feature = 'clean_text2'

In [36]:
features = df[text_feature].copy()
target = df['toxic'].copy()

In [37]:
X_train, X_rem, y_train, y_rem = train_test_split(features,target,  test_size=0.3, 
                              random_state=rstate, shuffle=True, stratify=df['toxic']  )
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5,  random_state=rstate, shuffle=True, stratify=y_rem )

Добавим выборки в пулы, более удобный формат Catboost'а:

In [39]:
X_test = X_test.to_frame()
X_val = X_val.to_frame()
X_train = X_train.to_frame()

train_pool = Pool(
    data=X_train,
    label=y_train,
    text_features=[text_feature]
)
valid_pool = Pool(
    data=X_val, 
    label=y_val,
    text_features=[text_feature],
)
test_pool = Pool(
    data=X_test, 
    label=y_test,
    text_features=[text_feature],
)

Обучаем модель:

In [40]:
model.fit(train_pool, 
          eval_set=valid_pool,
          early_stopping_rounds=50,
          verbose=100,
          # logging_level='Silent', 
          use_best_model=True )

Learning rate set to 0.050412
0:	learn: 0.6488544	test: 0.6482362	best: 0.6482362 (0)	total: 376ms	remaining: 31m 19s
100:	learn: 0.2917067	test: 0.2822269	best: 0.2822269 (100)	total: 32.8s	remaining: 26m 31s
200:	learn: 0.2703029	test: 0.2653748	best: 0.2653748 (200)	total: 1m 3s	remaining: 25m 26s
300:	learn: 0.2520823	test: 0.2526657	best: 0.2526657 (300)	total: 1m 35s	remaining: 24m 49s
400:	learn: 0.2391690	test: 0.2464182	best: 0.2464182 (400)	total: 2m 8s	remaining: 24m 31s
500:	learn: 0.2294852	test: 0.2428954	best: 0.2428954 (500)	total: 2m 39s	remaining: 23m 54s
600:	learn: 0.2214208	test: 0.2402659	best: 0.2402263 (598)	total: 3m 10s	remaining: 23m 14s
700:	learn: 0.2143903	test: 0.2385282	best: 0.2385282 (700)	total: 3m 41s	remaining: 22m 38s
800:	learn: 0.2084584	test: 0.2374647	best: 0.2374637 (799)	total: 4m 12s	remaining: 22m 2s
900:	learn: 0.2030327	test: 0.2362273	best: 0.2362273 (900)	total: 4m 42s	remaining: 21m 26s
1000:	learn: 0.1980572	test: 0.2355025	best: 0.23

Получим предсказания и метрики по ним:

In [41]:
val_preds = model.predict_proba(valid_pool)

In [42]:
print('val roc_auc', round( roc_auc_score(y_val, val_preds[:,1] ),3 ))

val roc_auc 0.969


In [43]:
f1_score(y_val, model.predict(valid_pool))

0.7233599444637279

Попробуем найти наилучший порог для получения максимального значения f1 на валидационной выборке:

In [44]:
def objective(trial, val_preds):
    threshold = trial.suggest_float("threshold", 0, 1)
    preds = (val_preds[:,1]> threshold) * 1

    return f1_score(y_val, preds)

In [45]:
study = optuna.create_study(
    direction="maximize", study_name="tresh", sampler=sampler
)
func = lambda trial: objective(trial, val_preds)
study.optimize(func, n_trials=100, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

In [46]:
print(f"Лучший показатель f1 на валидационной выборке: {study.best_value:.5f}")
print("")
print(f"Лучший порог:")

for key, value in study.best_params.items():
    print(f"{key}: {value}")
    best_tresh = value

Лучший показатель f1 на валидационной выборке: 0.77846

Лучший порог:
threshold: 0.7569209059583127


### Toxic bert

Попробуем модель, которая была создана для определения токсичных комментариев - Toxic-bert

Сделаем небольшую преобработку:

In [47]:
def slightly_text_preprocessing(text):
  text_only = re.sub('\n', ' ', text)
  final = ' '.join(text_only.split())
  return final

In [48]:
df['slightly_clean_text'] = df['text'].apply(slightly_text_preprocessing)

In [49]:
df.head()

,text,toxic,clean_text,clean_text2,slightly_clean_text
0,Explanation\nWhy the edits made under my usern...,0,explanation edits made username hardcore metal...,explanation edit make username hardcore metall...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,aww ! matches background colour seemingly stuc...,aww ! match background colour seemingly stuck ...,D'aww! He matches this background colour I'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man really trying edit war . guy constantl...,hey man really try edit war . guy constantly r...,"Hey man, I'm really not trying to edit war. It..."
3,"""\nMore\nI can't make any real suggestions on ...",0,make real suggestions improvement wondered sec...,make real suggestion improvement wonder sectio...,""" More I can't make any real suggestions on im..."
4,"You, sir, are my hero. Any chance you remember...",0,sir hero . chance remember page,sir hero . chance remember page,"You, sir, are my hero. Any chance you remember..."


Инициализируем модель и токенезатор:

In [52]:
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Токенизируем все данные:

In [53]:
tokenized = df['slightly_clean_text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))
n = max(len(x) for x in tokenized)
padded = np.array([i + [0]*(n - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
%%capture
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):

  batch = torch.from_numpy(padded[batch_size*i:batch_size*(i+1)])
  batch = torch.LongTensor(batch) 
  batch = batch.to(device)
  
  attention_mask_batch =  attention_mask[batch_size*i:batch_size*(i+1)]
  attention_mask_batch = torch.LongTensor(attention_mask_batch) 
  attention_mask_batch = attention_mask_batch.to(device)
  
  with torch.no_grad():
      batch_embeddings = model(batch, attention_mask=attention_mask_batch)
  
  embeddings.append( batch_embeddings[0][:,0,:].detach().cpu().numpy() )

  0%|          | 0/1592 [00:00<?, ?it/s]

In [54]:
# использование сохраненных эмбедингов

# import pickle
# embeddings_p = open ("/content/df_tweets_embeddings_sl_cleaned_toxic_bert.pkl", "rb")
# embeddings = pickle.load(embeddings_p)

In [55]:
features_e = np.concatenate(embeddings)
target_e = df['toxic']
target_e = target[:features_e.shape[0]]

Разделяем полученные данные на выборки:

In [56]:
X_train, X_rem, y_train, y_rem = train_test_split(features_e,target_e,  test_size=0.3, 
                              random_state=rstate, shuffle=True, stratify=target_e )
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5,  random_state=rstate, shuffle=True, stratify=y_rem )

Обучим логистическую регрессию на тренировочных эмбедингах:

In [57]:
%%capture
model = LogisticRegression( class_weight='balanced'   )
start = time.time()

model.fit(X_train, y_train          )
end = time.time()
fit_time = end - start

Получим предсказания на валидационной выборке и посчитаем метрику roc_auc:

In [59]:
ppreds_val = model.predict_proba(X_val)

In [60]:
print('val roc_auc', round( roc_auc_score(y_val, ppreds_val[:,1]) , 3) )

val roc_auc 0.997


Подберем оптимальный порог нашей модели для получения максимального значения метрики f1 на валидационной выборке:


In [61]:
def objective(trial, y_true, y_pred):
    threshold = trial.suggest_float("threshold", 0, 1)
    y_preds = (y_pred> threshold) * 1

    return f1_score(y_true, y_preds)

In [62]:
study = optuna.create_study(
    direction="maximize", study_name="tresh", sampler=sampler
)
func = lambda trial: objective(trial, y_val, ppreds_val[:,1])
study.optimize(func, n_trials=75, show_progress_bar=True)

  0%|          | 0/75 [00:00<?, ?it/s]

In [63]:
print(f"Лучший показатель f1 на валидационной выборке: {study.best_value:.5f}")
print("")
print(f"Лучший порог:")

for key, value in study.best_params.items():
    print(f"{key}: {value}")
    best_tresh = value

Лучший показатель f1 на валидационной выборке: 0.94268

Лучший порог:
threshold: 0.8925241796921707


Сохраним модель:

In [64]:
lr_bert = deepcopy(model)

По метрике f1 эта модель показывает наилучший результат. Значит, берем её для финального тестирования.

## Тестирование

Так как модель на основе bert имеет больший показатель roc_auc, то она проходит на финальный этап тестирования.

Делаем предсказания, считаем метрики лучшей модели на тестовых данных.

Считаем f1 на тестовой выборке:

In [65]:
test_preds = lr_bert.predict_proba(X_test)
test_predss = ( test_preds[:,1] > study.best_params['threshold'] ) * 1
round( f1_score(y_test, test_predss) , 3)

0.948

Считаем roc_auc на тестовой выборке:

In [66]:
print('test roc_auc', round( roc_auc_score(y_test, test_preds[:,1]) , 3) )

test roc_auc 0.998


 ## Выводы

Задача требовала построить модель позволяющую определять токсичные комментарии. 

Мы попробовали 3 метода решения этой задачи  для моделей - преобразования данных через TF-IDF, обучение алгоритма Catboost на текстовых данных и обучение логистической регрессии на эмбедингах Bert. Модель на основе Bert показала лучшие результаты по метрике roc_auc.

Результаты на тестовой выборке этой модели не сильно отличаются от того, что мы получили на валидационной, что говорит о надежности модели.

Подобрав оптимальный порог мы смогли получить значение метрики f1 равное 0.949,
что выше требуемого порога в 0.75

In [67]:
end_time_project = time.time()
time_of_proj = end_time_project - start_time_project
time_of_proj
print(f"Время выполнения проекта: {time_of_proj/60:.1f} минут")

Время выполнения проекта: 31.5 минут
